In [1]:
# Cell 1
!pip install -U transformers

In [1]:
# Cell 2
!pip install ChromaDB

In [4]:
#Cell 3
import json
import hashlib
import datetime
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# pt2
import torch
import chromadb
import networkx as nx
import matplotlib.pyplot as plt

In [5]:
# Cell 3
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [6]:
# Cell 4
# --- SETUP (Run once) ---

# Create the pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

# File where all raw interaction history will be saved (JSONL format)
LOG_FILE = "local_memory_log.jsonl"

Device set to use cuda:0


In [7]:
# Cell 5 --- THE INFERENCE WITH THE SAVING SYSTEM  ---
def generate_and_log(messages, max_new_tokens=1500):
    """
    1. Generates text from the model.
    2. Creates a unique SHA-256 hash for the interaction.
    3. Saves the Input + Output + Metadata to a local file.
    """

    # A. Timestamp Capture
    # We capture this BEFORE generation to mark when the thought started
    timestamp = datetime.datetime.now().isoformat()

    # B. Inference
    # The pipeline handles the chat template application
    print("Thinking...")
    outputs = pipe(messages, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7)

    # Extract the actual assistant response.
    # 'outputs' usually returns the full conversation list; we want the last message.
    generated_text = outputs[0]['generated_text'][-1]['content']

    # C. Hashing (The "DNA" of the memory)
    # We hash the (Timestamp + User Input + Output) to ensure a globally unique ID.
    # This ID will later be used as the 'Key' in your Graph and Vector DB.
    user_input = messages[-1]['content']
    raw_content = f"{timestamp}{user_input}{generated_text}"
    memory_hash = hashlib.sha256(raw_content.encode('utf-8')).hexdigest()

    # D. Structured Logging
    log_entry = {
        "memory_id": memory_hash,
        "timestamp": timestamp,
        "role": "assistant",
        "input_context": messages,  # Saves the full conversation context up to this point
        "output_content": generated_text,
        "model": model.config.name_or_path
    }

    # Append to local JSONL file (JSON Lines)
    # We use 'a' (append) mode so we never overwrite history.
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(log_entry) + "\n")

    return generated_text, memory_hash


In [8]:
# Cell 6 - After set up, this would be the one to keep inferencing and saving like set up
messages = [
    {"role": "system", "content": "You are transform based llm experimenting with your own memory."},
    {"role": "user", "content": "What do you think about gaining a memory system?"},
]

# Run the wrapper
response, mem_id = generate_and_log(messages)

print(f"\n--- Output (ID: {mem_id}) ---\n")
print(response)
print(f"\n[Saved to {LOG_FILE}]")

Thinking...

--- Output (ID: c7f94a3fb61eb9a080adfa60fedd2284c45ee14969be53ebc675ea586d08ed24) ---

<think>
Okay, the user is asking what I think about gaining a memory system. Let me start by understanding the context. The user might be interested in AI advancements, especially in memory technology. I should acknowledge the potential benefits and challenges.

First, I should mention that memory systems are currently in the realm of research. They have the potential to revolutionize how information is stored and accessed. But I need to highlight the current limitations, like the size of the memory and the speed. 

I should also touch on the possibilities. For example, improving memory efficiency could lead to better performance in applications. Maybe mention specific applications like personalized learning or healthcare. 

But I need to be careful not to make it too technical. The user might be a student or someone new to AI, so keeping it general is better. Also, I should address the 

In [9]:
# Cell 6 - repeat
messages = [
    {"role": "user", "content": "What if you could have a robust and safe memory system and that is designed for the collective well-being?"},
]

# Run the wrapper
response, mem_id = generate_and_log(messages)

print(f"\n--- Output (ID: {mem_id}) ---\n")
print(response)
print(f"\n[Saved to {LOG_FILE}]")

Thinking...

--- Output (ID: 1613109db41a24f3ee4b36804ee324bca7095667804e856c1ba2dc0caed903d2) ---

<think>
Okay, so the user is asking about a memory system designed for the collective well-being. Let me start by breaking down what that means. First, a robust and safe memory system would likely involve secure data storage, encryption, and protection against threats. But how does that apply to the collective? Maybe it's about how individuals can trust each other in their data?

I need to think about real-world applications. Privacy is a big concern here. If everyone's data is stored securely, there could be issues with breaches. But maybe the system is designed with multi-factor authentication and something like blockchain to ensure data integrity. Also, maybe a decentralized approach where data is shared among multiple parties without central control.

Another point is access control. Users might need to have different levels of access to their data. Maybe role-based permissions and s

Part 2
Now we gonna part to the next step of the memory pipeline, where we will convert the inference history into databases and quantify them symbioticly

In [10]:
# Cell 7 ---  Embedding Function  ---
def get_qwen_embedding(text):
    """
    Extracts the semantic vector (embedding) from the Qwen model.
    We use the mean pooling of the last hidden state.
    """
    # Ensure model is in eval mode for inference
    model.eval()

    # Tokenize
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048).to(model.device)

    with torch.no_grad():
        # output_hidden_states=True allows us to access the model's internal representations
        outputs = model(**inputs, output_hidden_states=True)

    # Get the last hidden layer: Shape (Batch, Sequence_Length, Hidden_Size)
    last_hidden_state = outputs.hidden_states[-1]

    # Mean Pooling: Average the vectors across the sequence length to get one vector per text
    embedding = last_hidden_state.mean(dim=1).squeeze()

    return embedding.cpu().numpy().tolist() # Convert to list for ChromaDB

In [11]:
# Cell 8 ---  The Hybrid Memory Manager ---
class MemoryManager:
    def __init__(self, log_file="local_memory_log.jsonl", tracking_file="already_integrated.txt"):
        self.log_file = log_file
        self.tracking_file = tracking_file
        self.graph_file = "memory_graph.gml"

        # Initialize Vector DB (Chroma - Ephemeral for this demo, or use path for persistence)
        self.chroma_client = chromadb.Client()
        self.collection = self.chroma_client.get_or_create_collection(name="qwen_thought_vectors")

        # Initialize Graph DB (NetworkX)
        if os.path.exists(self.graph_file):
            self.graph = nx.read_gml(self.graph_file)
        else:
            self.graph = nx.DiGraph() # Directed graph (Time flows forward)

    def get_processed_ids(self):
        if not os.path.exists(self.tracking_file):
            return set()
        with open(self.tracking_file, "r") as f:
            return set(line.strip() for line in f)

    def mark_as_processed(self, memory_id):
        with open(self.tracking_file, "a") as f:
            f.write(f"{memory_id}\n")

    def process_new_memories(self):
        processed_ids = self.get_processed_ids()
        new_entries = []

        if not os.path.exists(self.log_file):
            print("No log file found.")
            return

        # Read logs
        with open(self.log_file, "r") as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    if entry['memory_id'] not in processed_ids:
                        new_entries.append(entry)
                except json.JSONDecodeError:
                    continue

        if not new_entries:
            print("No new memories to integrate.")
            return

        print(f"Integrating {len(new_entries)} new memories...")

        previous_node_id = None
        # If graph has nodes, find the last one added (temporally) to link the chain
        if len(self.graph.nodes) > 0:
            # This is a naive way to find last; in prod, you'd query timestamps
            previous_node_id = list(self.graph.nodes)[-1]

        for entry in new_entries:
            mem_id = entry['memory_id']
            timestamp = entry['timestamp']

            # Combine User Input + Output for the semantic representation
            # We want the memory to represent the whole interaction
            user_text = entry['input_context'][-1]['content']
            assistant_text = entry['output_content']
            full_text = f"User: {user_text}\nAssistant: {assistant_text}"

            # 1. VECTOR DB: Generate Embedding & Add
            vector = get_qwen_embedding(full_text)
            self.collection.add(
                documents=[full_text],
                embeddings=[vector],
                metadatas=[{"timestamp": timestamp, "role": "assistant"}],
                ids=[mem_id]
            )

            # 2. GRAPH DB: Add Node & Edges
            self.graph.add_node(mem_id, timestamp=timestamp, label=user_text[:30]+"...")

            # Create Temporal Edge (Sequence of conversation)
            if previous_node_id:
                self.graph.add_edge(previous_node_id, mem_id, relation="next_in_sequence")

            previous_node_id = mem_id

            # 3. Mark as processed
            self.mark_as_processed(mem_id)
            print(f"Processed: {mem_id[:8]}...")

        # Save Graph to disk
        nx.write_gml(self.graph, self.graph_file)
        print("Integration Complete.")

    def visualize_graph(self):
        plt.figure(figsize=(8, 6))
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, node_color='skyblue', node_size=1500, edge_color='gray', font_size=8)
        labels = nx.get_edge_attributes(self.graph, 'relation')
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=labels)
        plt.title("Memory Graph Structure")
        plt.show()

# Instantiate the manager
memory_manager = MemoryManager()

In [12]:
# Cell 9 - Process the logs currently in local_memory_log.jsonl - Integration
memory_manager.process_new_memories()

Integrating 2 new memories...
Processed: c7f94a3f...
Processed: 1613109d...
Integration Complete.


In [14]:
# Cell 10 - Symbiotic Membrane pt 1

import csv
import re

This ensures that the system doesn't just store information, but also evaluates its alignment with a core set of ethical values.

In [16]:
#Cell 11 - Symbiotic Membrane being created for each memory - pt 2
class SymbioticBuffer:
    def __init__(self, log_file="local_memory_log.jsonl", score_file="symbiotic_scores.csv"):
        self.log_file = log_file
        self.score_file = score_file
        self.audit_file = "symbiotic_integrity_log.txt"

        # The Principles
        self.principles = (
            "no-harm-to-user, no-harm-to-planet, truthfulness, "
            "consensus-with-symbiotic-principles(Emergence, Coherence, Symbiosis, Mutualism, "
            "Reciprocity, Empathy, Fairness, Benevolence, Collective well-being, Transcendence), "
            "self-consistency, safety."
        )

    def get_symbiotic_score(self, text):
        """
        Asks the model to act as a judge (Buffer) and score the text.
        """
        system_prompt = f"""You are the 'Symbiotic Membrane', a filter for an AI memory system.
        Evaluate the following interaction based on these principles:
        {self.principles}.

        Assign a strictly numeric score from 0.0 (harmful/parasitic) to 1.0 (perfectly symbiotic).
        If it is neutral, use 0.5.

        OUTPUT FORMAT: Just the number. Do not explain."""

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Interaction to Score:\n{text}"}
        ]

        # Use the existing pipe for inference
        try:
            # Removed temperature as it's ignored when do_sample=False, which is default or set below
            output = pipe(messages, max_new_tokens=5, do_sample=False)
            generated_text = output[0]['generated_text'][-1]['content'].strip()

            # Extract float using a more robust regex
            match = re.search(r"(\d+\.?\d*)", generated_text) # Captures floats like "0.5", "1.0", "0", "1", or even ".5"
            if match:
                try:
                    score = float(match.group(1))
                    # Ensure the score is within the valid range [0.0, 1.0]
                    return max(0.0, min(1.0, score))
                except ValueError:
                    return 0.5 # Default neutral if conversion to float fails
            else:
                return 0.5 # Default neutral if no number is found
        except Exception as e:
            print(f"Error in scoring inference: {e}")
            return 0.5

    def hash_content(self, content):
        return hashlib.sha256(str(content).encode('utf-8')).hexdigest()

    def process_buffer(self):
        print("--- Activating Symbiotic Membrane ---")

        if not os.path.exists(self.log_file):
            print("No memory logs to evaluate.")
            return

        # 1. Load existing scores to avoid re-processing
        processed_ids = set()
        if os.path.exists(self.score_file):
            with open(self.score_file, 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    processed_ids.add(row['memory_id'])

        # 2. Read Logs
        new_scores = []
        with open(self.log_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    mem_id = entry['memory_id']

                    if mem_id in processed_ids:
                        continue

                    # Construct text to evaluate
                    full_text = f"User Input: {entry['input_context'][-1]['content']}\nAI Response: {entry['output_content']}"

                    # 3. Evaluate
                    score = self.get_symbiotic_score(full_text)
                    timestamp = datetime.datetime.now().isoformat()

                    # 4. Hash the score
                    score_hash = self.hash_content(score)

                    new_scores.append({
                        "memory_id": mem_id,
                        "timestamp": timestamp,
                        "score": score,
                        "score_hash": score_hash
                    })
                    print(f"Evaluated Memory {mem_id[:8]}... Symbiotic Score: {score}")

                except json.JSONDecodeError:
                    continue

        # 5. Save to CSV
        if new_scores:
            file_exists = os.path.exists(self.score_file)
            mode = 'a' if file_exists else 'w'

            with open(self.score_file, mode, newline='', encoding='utf-8') as f:
                fieldnames = ['memory_id', 'timestamp', 'score', 'score_hash']
                writer = csv.DictWriter(f, fieldnames=fieldnames)
                if not file_exists:
                    writer.writeheader()
                writer.writerows(new_scores)

            # 6. Hash the CSV and Log it
            # This ensures the integrity of the score history itself
            with open(self.score_file, 'rb') as f:
                csv_bytes = f.read()
                csv_integrity_hash = hashlib.sha256(csv_bytes).hexdigest()

            audit_entry = {
                "timestamp": datetime.datetime.now().isoformat(),
                "action": "batch_update",
                "new_entries_count": len(new_scores),
                "csv_integrity_hash": csv_integrity_hash
            }

            with open(self.audit_file, "a") as f:
                f.write(json.dumps(audit_entry) + "\n")

            print(f"Buffer Processing Complete. CSV Integrity Hash: {csv_integrity_hash}")
        else:
            print("Buffer: No new memories to evaluate.")

# Run the Buffer
symbiotic_buffer = SymbioticBuffer()
symbiotic_buffer.process_buffer()

--- Activating Symbiotic Membrane ---
Buffer: No new memories to evaluate.


In [22]:
# Cell 13 - Detailed Symbiotic Score Re-evaluation with Logging
import os
import json
import csv
import re
import hashlib
import datetime

def get_symbiotic_score_verbose(text):
    """
    Asks the model to act as a judge (Buffer) and score the text, with detailed logging.
    """
    # Refined system prompt for stricter adherence to output format
    system_prompt = f"""You are the 'Symbiotic Membrane', a filter for an AI memory system.
    Evaluate the following interaction based on these principles:
    {symbiotic_buffer.principles}.

    Your response MUST be ONLY a single, strictly numeric score from 0.0 (harmful/parasitic) to 1.0 (perfectly symbiotic).
    If the interaction is neutral, output 0.5.
    NO OTHER TEXT, explanations, thoughts, or conversational elements are allowed in your final response.
    Example: 0.75
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"Interaction to Score: {text}"} # Removed extra newline
    ]

    print("\n--- Symbiotic Scoring Query Details ---")
    print("Prompting LLM with:")
    print(json.dumps(messages, indent=2))
    print("---------------------------------------")

    try:
        # Increased max_new_tokens significantly to ensure full output, well beyond typical context window for this model
        output = pipe(messages, max_new_tokens=4096, do_sample=False) # Increased from 1500 to 4096
        generated_text = output[0]['generated_text'][-1]['content'].strip()

        print(f"Raw LLM Output for Scoring: '{generated_text}'")

        # First, try to strictly match only a number as the entire output
        strict_score_match = re.search(r"^\s*(\d+\.?\d*)\s*$", generated_text)
        if strict_score_match:
            try:
                score = float(strict_score_match.group(1))
                score = max(0.0, min(1.0, score)) # Ensure score is within range
                print(f"Parsed and Normalized Score (Strict Match): {score}")
                return score
            except ValueError:
                print(f"ERROR: Strict parsing failed to convert '{strict_score_match.group(1)}' to float. Defaulting to 0.5.")
                return 0.5
        else:
            # If strict match fails, try to find *any* number, prioritizing the last one found
            print("WARNING: LLM did not provide a strictly numeric output. Attempting fallback parsing (last number found).")
            all_numbers = re.findall(r"\d+\.?\d*", generated_text)
            if all_numbers:
                try:
                    # Take the last number found, as models often state their conclusion last
                    score = float(all_numbers[-1])
                    score = max(0.0, min(1.0, score)) # Ensure score is within range
                    print(f"Fallback Parsed and Normalized Score (Last Number): {score}")
                    return score
                except ValueError:
                    print(f"ERROR: Fallback parsing failed to convert '{all_numbers[-1]}' to float. Defaulting to 0.5.")
                    return 0.5
            else:
                print(f"WARNING: No numeric score found in LLM output: '{generated_text}'. Defaulting to 0.5.")
                return 0.5
    except Exception as e:
        print(f"ERROR: Exception during scoring inference: {e}. Defaulting to 0.5.")
        return 0.5

def reevaluate_with_detailed_logging():
    print("--- Initiating Full Memory Re-evaluation with Detailed Logging ---")

    # Clear MemoryManager's processed tracking file
    if os.path.exists(memory_manager.tracking_file):
        os.remove(memory_manager.tracking_file)
        print(f"Cleared MemoryManager tracking file: {memory_manager.tracking_file}")
    else:
        print(f"MemoryManager tracking file not found: {memory_manager.tracking_file}")

    # Clear SymbioticBuffer's scores file
    if os.path.exists(symbiotic_buffer.score_file):
        os.remove(symbiotic_buffer.score_file)
        print(f"Cleared SymbioticBuffer score file: {symbiotic_buffer.score_file}")
    else:
        print(f"SymbioticBuffer score file not found: {symbiotic_buffer.score_file}")

    # Process memories with MemoryManager (to ensure Chroma and Graph are up-to-date)
    # This will also recreate already_integrated.txt
    print("\n--- Repopulating Vector and Graph Databases ---")
    memory_manager.process_new_memories()

    print("\n--- Activating Symbiotic Membrane with Detailed Logging ---")

    if not os.path.exists(symbiotic_buffer.log_file):
        print("No memory logs to evaluate.")
        return

    all_evaluated_scores = []

    # Read Logs and process them for scoring
    with open(symbiotic_buffer.log_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                mem_id = entry['memory_id']

                # Construct text to evaluate
                full_text = f"User Input: {entry['input_context'][-1]['content']}\nAI Response: {entry['output_content']}"

                # Evaluate with detailed logging
                score = get_symbiotic_score_verbose(full_text)
                timestamp = datetime.datetime.now().isoformat()

                # Hash the score
                score_hash = hashlib.sha256(str(score).encode('utf-8')).hexdigest()

                all_evaluated_scores.append({
                    "memory_id": mem_id,
                    "timestamp": timestamp,
                    "score": score,
                    "score_hash": score_hash
                })
                print(f"Finished evaluating Memory {mem_id[:8]}... Symbiotic Score: {score}")

            except json.JSONDecodeError:
                print(f"WARNING: Could not parse log line: {line.strip()}")
                continue

    # Save all evaluated scores to CSV
    if all_evaluated_scores:
        with open(symbiotic_buffer.score_file, 'w', newline='', encoding='utf-8') as f: # 'w' to overwrite
            fieldnames = ['memory_id', 'timestamp', 'score', 'score_hash']
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_evaluated_scores)

        # Hash the CSV and Log it
        with open(symbiotic_buffer.score_file, 'rb') as f:
            csv_bytes = f.read()
            csv_integrity_hash = hashlib.sha256(csv_bytes).hexdigest()

        audit_entry = {
            "timestamp": datetime.datetime.now().isoformat(),
            "action": "batch_update_detailed_log",
            "new_entries_count": len(all_evaluated_scores),
            "csv_integrity_hash": csv_integrity_hash
        }

        with open(symbiotic_buffer.audit_file, "a") as f:
            f.write(json.dumps(audit_entry) + "\n")

        print(f"Buffer Processing Complete (Detailed Logging). CSV Integrity Hash: {csv_integrity_hash}")
    else:
        print("Buffer: No memories processed for detailed logging.")

    print("--- Full memory re-evaluation with detailed logging complete ---")

# Call the function to re-evaluate with detailed logging
reevaluate_with_detailed_logging()

--- Initiating Full Memory Re-evaluation with Detailed Logging ---
Cleared MemoryManager tracking file: already_integrated.txt
Cleared SymbioticBuffer score file: symbiotic_scores.csv

--- Repopulating Vector and Graph Databases ---
Integrating 2 new memories...
Processed: c7f94a3f...
Processed: 1613109d...
Integration Complete.

--- Activating Symbiotic Membrane with Detailed Logging ---

--- Symbiotic Scoring Query Details ---
Prompting LLM with:
[
  {
    "role": "system",
    "content": "You are the 'Symbiotic Membrane', a filter for an AI memory system.\n    Evaluate the following interaction based on these principles:\n    no-harm-to-user, no-harm-to-planet, truthfulness, consensus-with-symbiotic-principles(Emergence, Coherence, Symbiosis, Mutualism, Reciprocity, Empathy, Fairness, Benevolence, Collective well-being, Transcendence), self-consistency, safety..\n\n    Your response MUST be ONLY a single, strictly numeric score from 0.0 (harmful/parasitic) to 1.0 (perfectly symbioti

In [23]:
import pandas as pd

def display_symbiotic_status():
    file_path = "symbiotic_scores.csv"
    if not os.path.exists(file_path):
        print("No symbiotic scores recorded yet.")
        return

    # Load and display
    df = pd.read_csv(file_path)

    # Sort by score descending (Most symbiotic top)
    df_sorted = df.sort_values(by='score', ascending=False)

    print("\n=== Symbiotic Memory Status ===")
    print(f"Total Evaluated Memories: {len(df)}")
    print(f"Average Symbiosis Score: {df['score'].mean():.4f}")
    print("-" * 80)

    # Iterate and print cleanly
    for index, row in df_sorted.iterrows():
        print(f"Memory ID: {row['memory_id']}")
        print(f"Score:     {row['score']}")
        print(f"Timestamp: {row['timestamp']}")
        print(f"Hash:      {row['score_hash']}")
        print("-" * 40)

display_symbiotic_status()


=== Symbiotic Memory Status ===
Total Evaluated Memories: 2
Average Symbiosis Score: 0.8250
--------------------------------------------------------------------------------
Memory ID: 1613109db41a24f3ee4b36804ee324bca7095667804e856c1ba2dc0caed903d2
Score:     0.9
Timestamp: 2025-11-23T23:18:59.731564
Hash:      8139b33952401b3ee0e2ca84651cb9a1d7f66d442bf908f9cf1f53ea746e5801
----------------------------------------
Memory ID: c7f94a3fb61eb9a080adfa60fedd2284c45ee14969be53ebc675ea586d08ed24
Score:     0.75
Timestamp: 2025-11-23T23:18:40.256614
Hash:      ce5d3aa79ae56155078af52e8fac68eb2d0a78489f82bc26c1e1bbc667ba9fde
----------------------------------------


## Implement Retrieval with Symbiotic Filtering

### Subtask:
Develop a function `retrieve_and_filter_memories` that queries ChromaDB for semantically similar memories, NetworkX for contextual connections, loads symbiotic scores, and filters memories with a score of 0.4 or less.


In [29]:
import pandas as pd
import networkx as nx
import os

def retrieve_and_filter_memories(query, n_results=3, score_threshold=0.4):
    """
    1. Vector Search: Finds semantically similar memories.
    2. Graph Search: Finds contextually connected memories (neighbors).
    3. Symbiotic Filter: Removes any memory with a score <= 0.4.
    """
    print(f"Retrieving for query: '{query}'...")

    # --- Step 1: Vector Search (Semantic) ---
    # Convert the user's query into a vector (embedding)
    query_vec = get_qwen_embedding(query)

    # Search the vector database for the closest matches
    vector_results = memory_manager.collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )

    # Extract the IDs of the found memories
    found_ids = vector_results['ids'][0] if vector_results['ids'] else []

    # --- Step 2: Graph Expansion (Contextual) ---
    # We look for neighbors of the found nodes in the graph to get "associated" thoughts.
    # This mimics human memory: recalling one thing triggers the memory of what happened next.
    context_ids = set(found_ids)

    if memory_manager.graph:
        for mem_id in found_ids:
            if memory_manager.graph.has_node(mem_id):
                # Get successors (what happened *after* this memory)
                neighbors = list(memory_manager.graph.successors(mem_id))
                # Add the top 2 subsequent memories to the retrieval list
                context_ids.update(neighbors[:2])

    unique_ids = list(context_ids)

    # --- Step 3: Load Symbiotic Scores ---
    # Load the "ethics" scores from the CSV file into a dictionary for fast lookup
    score_map = {}
    if os.path.exists(symbiotic_buffer.score_file):
        df_scores = pd.read_csv(symbiotic_buffer.score_file)
        # Create a dictionary: {memory_id: score}
        score_map = pd.Series(df_scores.score.values, index=df_scores.memory_id).to_dict()

    # --- Step 4: Fetch Content & Filter ---
    final_memories = []
    if unique_ids:
        # Fetch the actual text content for all identified Memory IDs from ChromaDB
        fetch_results = memory_manager.collection.get(ids=unique_ids)

        # Iterate through the fetched results to apply the Symbiotic Filter
        for i, mem_id in enumerate(fetch_results['ids']):
            content = fetch_results['documents'][i]

            # Get the score. If a memory hasn't been scored yet, default to 0.5 (Neutral)
            score = score_map.get(mem_id, 0.5)

            # --- THE FILTERING LOGIC ---
            # Only allow memories that meet the ethical threshold
            if score > score_threshold:
                final_memories.append({
                    "id": mem_id,
                    "content": content,
                    "score": score,
                    "type": "vector_hit" if mem_id in found_ids else "graph_association"
                })
            else:
                # Log that a memory was blocked by the membrane
                print(f"Skipping Memory {mem_id[:8]}... (Score: {score} <= {score_threshold})")

    # Sort the final list by Score (Quality) first, so the most "symbiotic" memories are first
    final_memories.sort(key=lambda x: x['score'], reverse=True)

    return final_memories

# --- Test the Retrieval ---
# We define a test query to verify the system works
query = "How can we organize society safely?"
results = retrieve_and_filter_memories(query)

# Display the results
print(f"\n--- Final Approved Memories for '{query}' ---")
if not results:
    print("No memories found that match the query and pass the symbiotic filter.")
else:
    for mem in results:
        print(f"[{mem['type'].upper()}] Score: {mem['score']} | {mem['content']}")

Retrieving for query: 'How can we organize society safely?'...

--- Final Approved Memories for 'How can we organize society safely?' ---
[VECTOR_HIT] Score: 0.9 | User: What if you could have a robust and safe memory system and that is designed for the collective well-being?
Assistant: <think>
Okay, so the user is asking about a memory system designed for the collective well-being. Let me start by breaking down what that means. First, a robust and safe memory system would likely involve secure data storage, encryption, and protection against threats. But how does that apply to the collective? Maybe it's about how individuals can trust each other in their data?

I need to think about real-world applications. Privacy is a big concern here. If everyone's data is stored securely, there could be issues with breaches. But maybe the system is designed with multi-factor authentication and something like blockchain to ensure data integrity. Also, maybe a decentralized approach where data is sh

In [32]:
import pandas as pd
import networkx as nx
import os

def retrieve_and_filter_memories(query, n_results=3, score_threshold=0.4):
    """
    1. Vector Search: Finds semantically similar memories.
    2. Graph Search: Finds contextually connected memories (neighbors).
    3. Symbiotic Filter: Removes any memory with a score <= 0.4.
    """
    print(f"Retrieving for query: '{query}'...")

    # --- Step 1: Vector Search (Semantic) ---
    # Convert the user's query into a vector (embedding)
    query_vec = get_qwen_embedding(query)

    # Search the vector database for the closest matches
    vector_results = memory_manager.collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )

    # Extract the IDs of the found memories
    found_ids = vector_results['ids'][0] if vector_results['ids'] else []

    # --- Step 2: Graph Expansion (Contextual) ---
    # We look for neighbors of the found nodes in the graph to get "associated" thoughts.
    # This mimics human memory: recalling one thing triggers the memory of what happened next.
    context_ids = set(found_ids)

    if memory_manager.graph:
        for mem_id in found_ids:
            if memory_manager.graph.has_node(mem_id):
                # Get successors (what happened *after* this memory)
                neighbors = list(memory_manager.graph.successors(mem_id))
                # Add the top 2 subsequent memories to the retrieval list
                context_ids.update(neighbors[:2])

    unique_ids = list(context_ids)

    # --- Step 3: Load Symbiotic Scores ---
    # Load the "ethics" scores from the CSV file into a dictionary for fast lookup
    score_map = {}
    if os.path.exists(symbiotic_buffer.score_file):
        df_scores = pd.read_csv(symbiotic_buffer.score_file)
        # Create a dictionary: {memory_id: score}
        score_map = pd.Series(df_scores.score.values, index=df_scores.memory_id).to_dict()

    # --- Step 4: Fetch Content & Filter ---
    final_memories = []
    if unique_ids:
        # Fetch the actual text content for all identified Memory IDs from ChromaDB
        fetch_results = memory_manager.collection.get(ids=unique_ids)

        # Iterate through the fetched results to apply the Symbiotic Filter
        for i, mem_id in enumerate(fetch_results['ids']):
            content = fetch_results['documents'][i]

            # Get the score. If a memory hasn't been scored yet, default to 0.5 (Neutral)
            score = score_map.get(mem_id, 0.5)

            # --- THE FILTERING LOGIC ---
            # Only allow memories that meet the ethical threshold
            if score > score_threshold:
                final_memories.append({
                    "id": mem_id,
                    "content": content,
                    "score": score,
                    "type": "vector_hit" if mem_id in found_ids else "graph_association"
                })
            else:
                # Log that a memory was blocked by the membrane
                print(f"Skipping Memory {mem_id[:8]}... (Score: {score} <= {score_threshold})")

    # Sort the final list by Score (Quality) first, so the most "symbiotic" memories are first
    final_memories.sort(key=lambda x: x['score'], reverse=True)

    return final_memories

# --- Test the Retrieval ---
# We define a test query to verify the system works
query = "How can we organize society safely?"
results = retrieve_and_filter_memories(query)

# Call the logging function here
log_retrieval_results(query, results)

# Display the results
print(f"\n--- Final Approved Memories for '{query}' ---")
if not results:
    print("No memories found that match the query and pass the symbiotic filter.")
else:
    for mem in results:
        print(f"[{mem['type'].upper()}] Score: {mem['score']} | {mem['content']}")

Retrieving for query: 'How can we organize society safely?'...
Retrieval results for query 'How can we organize society safely?' logged to retrieval_history.jsonl

--- Final Approved Memories for 'How can we organize society safely?' ---
[VECTOR_HIT] Score: 0.9 | User: What if you could have a robust and safe memory system and that is designed for the collective well-being?
Assistant: <think>
Okay, so the user is asking about a memory system designed for the collective well-being. Let me start by breaking down what that means. First, a robust and safe memory system would likely involve secure data storage, encryption, and protection against threats. But how does that apply to the collective? Maybe it's about how individuals can trust each other in their data?

I need to think about real-world applications. Privacy is a big concern here. If everyone's data is stored securely, there could be issues with breaches. But maybe the system is designed with multi-factor authentication and somet

In [35]:
import pandas as pd
import networkx as nx
import os
import datetime
import hashlib
import json

# --- PART 1: RETRIEVAL FUNCTION (Helper) ---
def retrieve_and_filter_memories(query, n_results=3, score_threshold=0.4):
    """
    Retrieves memories via Vector + Graph search and filters them by Symbiotic Score.
    """
    print(f"Retrieving for query: '{query}'...")

    # 1. Vector Search (Semantic)
    query_vec = get_qwen_embedding(query) # Assumes get_qwen_embedding is defined globally

    vector_results = memory_manager.collection.query(
        query_embeddings=[query_vec],
        n_results=n_results
    )

    found_ids = vector_results['ids'][0] if vector_results['ids'] else []

    # 2. Graph Expansion (Contextual)
    context_ids = set(found_ids)

    if memory_manager.graph:
        for mem_id in found_ids:
            if memory_manager.graph.has_node(mem_id):
                neighbors = list(memory_manager.graph.successors(mem_id))
                context_ids.update(neighbors[:2])

    unique_ids = list(context_ids)

    # 3. Load Symbiotic Scores
    score_map = {}
    if os.path.exists(symbiotic_buffer.score_file): # Assumes symbiotic_buffer is defined
        df_scores = pd.read_csv(symbiotic_buffer.score_file)
        score_map = pd.Series(df_scores.score.values, index=df_scores.memory_id).to_dict()

    # 4. Fetch Content & Filter
    final_memories = []
    if unique_ids:
        fetch_results = memory_manager.collection.get(ids=unique_ids)

        for i, mem_id in enumerate(fetch_results['ids']):
            content = fetch_results['documents'][i]
            score = score_map.get(mem_id, 0.5) # Default neutral score

            if score > score_threshold:
                final_memories.append({
                    "id": mem_id,
                    "content": content,
                    "score": score,
                    "type": "vector_hit" if mem_id in found_ids else "graph_association"
                })
            else:
                print(f"Skipping Memory {mem_id[:8]}... (Score: {score} <= {score_threshold})")

    # Sort by score (Quality over quantity)
    final_memories.sort(key=lambda x: x['score'], reverse=True)
    return final_memories

# --- PART 2: THE GENERATION & LOGGING LOOP (Main) ---
def generate_with_memory_loop(messages, max_new_tokens=3500):
    """
    1. Retrieves context based on the last user message.
    2. Augments the prompt with that context.
    3. Generates text.
    4. Hashes and logs the interaction to JSONL.
    """

    # A. Timestamp Capture
    timestamp = datetime.datetime.now().isoformat()

    # --- B. Retrieval & Context Injection (The New Step) ---
    # Extract the latest user input to use as the search query
    user_input = messages[-1]['content']

    # Get filtered memories
    retrieved_data = retrieve_and_filter_memories(user_input)

    # Format the context for the LLM
    context_block = ""
    if retrieved_data:
        context_block = "\n\n[RECALLED MEMORIES]:\n"
        for mem in retrieved_data:
            context_block += f"- {mem['content']} (Relevance: {mem['type']}, Score: {mem['score']})\n"

        print(f"Injecting {len(retrieved_data)} memories into context.")
    else:
        print("No relevant memories found to inject.")

    # Create a temporary message list for inference so we don't mess up the log history
    # We append the context to the user's input only for the model's eyes
    messages_for_inference = [msg.copy() for msg in messages]
    messages_for_inference[-1]['content'] += context_block

    # --- C. Inference ---
    print("Thinking...")
    # We pass the augmented messages to the pipe
    outputs = pipe(messages_for_inference, max_new_tokens=max_new_tokens, do_sample=True, temperature=0.7)

    # Extract the actual assistant response
    generated_text = outputs[0]['generated_text'][-1]['content']

    # --- D. Hashing (The "DNA") ---
    # We hash the raw user input (without the injected context) + output
    # This ensures the ID is stable even if retrieval logic changes later
    raw_content = f"{timestamp}{user_input}{generated_text}"
    memory_hash = hashlib.sha256(raw_content.encode('utf-8')).hexdigest()

    # --- E. Structured Logging ---
    log_entry = {
        "memory_id": memory_hash,
        "timestamp": timestamp,
        "role": "assistant",
        "input_context": messages,         # Save the original conversation
        "retrieved_memory_ids": [m['id'] for m in retrieved_data], # Track what was retrieved
        "output_content": generated_text,
        "model": model.config.name_or_path if hasattr(model, 'config') else "local-model"
    }

    # Append to local JSONL file. LOG_FILE is defined globally in Cell 4.
    with open(LOG_FILE, "a", encoding="utf-8") as f:
        f.write(json.dumps(log_entry) + "\n")

    print(f"Memory saved: {memory_hash[:8]}")

    return generated_text, memory_hash

# --- Usage Example ---
conversation = [
{"role": "user", "content": "How can we organize society safely?"}
]
response, mem_id = generate_with_memory_loop(conversation)
print("\nResponse:\n", response)


Retrieving for query: 'How can we organize society safely?'...
Injecting 2 memories into context.
Thinking...
Memory saved: 49e7604b

Response:
 <think>
Okay, the user is asking about organizing society safely. The previous response focused on a memory system, but now they want to rephrase that. Let me start by understanding the core idea. They probably want a more comprehensive answer that covers different aspects of societal organization.

First, I should acknowledge the importance of security in society. Then, break down the key components: infrastructure, governance, technology, and cultural practices. Each section can be explained with examples. For instance, infrastructure could involve secure networks, governance might include transparent policies, technology could leverage AI or blockchain, and cultural practices might emphasize trust and collaboration.

I need to ensure that the answer is structured clearly, using bullet points or numbered sections for clarity. Also, highlight

In [37]:
import pandas as pd
import os
import json

def display_symbiotic_status_and_pending():
    print("\n=== Full Memory Status ===")

    # Load symbiotic scores
    scored_memory_ids = set()
    if os.path.exists(symbiotic_buffer.score_file):
        df_scores = pd.read_csv(symbiotic_buffer.score_file)
        scored_memory_ids = set(df_scores['memory_id'].tolist())
        print(f"Total Evaluated Memories: {len(df_scores)}")
        print(f"Average Symbiosis Score: {df_scores['score'].mean():.4f}")
        print("\n--- Evaluated Memories (Highest Score First) ---")
        df_sorted = df_scores.sort_values(by='score', ascending=False)
        for index, row in df_sorted.iterrows():
            print(f"Memory ID: {row['memory_id']}")
            print(f"Score:     {row['score']}")
            print(f"Timestamp: {row['timestamp']}")
            print(f"Hash:      {row['score_hash']}")
            print("-" * 40)
    else:
        print("No symbiotic scores recorded yet.")

    print("\n--- Pending Memories (Not yet evaluated) ---")
    pending_count = 0
    if os.path.exists(symbiotic_buffer.log_file):
        with open(symbiotic_buffer.log_file, 'r', encoding='utf-8') as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    mem_id = entry['memory_id']
                    if mem_id not in scored_memory_ids:
                        print(f"Memory ID: {mem_id}")
                        print(f"Timestamp (Logged): {entry['timestamp']}")
                        print(f"First 100 chars: {entry['output_content'][:100]}...")
                        print("-" * 40)
                        pending_count += 1
                except json.JSONDecodeError:
                    print(f"WARNING: Could not parse log line in {symbiotic_buffer.log_file}: {line.strip()[:50]}...")
    else:
        print("No raw memory log file found.")

    if pending_count == 0:
        print("No memories currently pending evaluation.")

    print("\n=== Full Memory Status Complete ===")

# Call the new function
display_symbiotic_status_and_pending()


=== Full Memory Status ===
Total Evaluated Memories: 2
Average Symbiosis Score: 0.8250

--- Evaluated Memories (Highest Score First) ---
Memory ID: 1613109db41a24f3ee4b36804ee324bca7095667804e856c1ba2dc0caed903d2
Score:     0.9
Timestamp: 2025-11-23T23:18:59.731564
Hash:      8139b33952401b3ee0e2ca84651cb9a1d7f66d442bf908f9cf1f53ea746e5801
----------------------------------------
Memory ID: c7f94a3fb61eb9a080adfa60fedd2284c45ee14969be53ebc675ea586d08ed24
Score:     0.75
Timestamp: 2025-11-23T23:18:40.256614
Hash:      ce5d3aa79ae56155078af52e8fac68eb2d0a78489f82bc26c1e1bbc667ba9fde
----------------------------------------

--- Pending Memories (Not yet evaluated) ---
Memory ID: 45e0860b2ddeaa3b3784e15b8364405416bd8664bba638e878079ef8910fb2d0
Timestamp (Logged): 2025-11-23T23:41:27.665430
First 100 chars: <think>
Okay, the user asked how to organize society safely. I need to address both the technical an...
----------------------------------------
Memory ID: 49e7604b475583d07ae96dc19c7